In [ ]:
import torch
from brepmatching.utils import zip_hetdata
from automate import Part, PartFeatures, part_to_graph, PartOptions
import json
import xxhash
from brepmatching.visualization import render_predictions, show_image

In [ ]:
p0 = Part('/home/ben/Downloads/output_baseline1.x_t')
p1 = Part('/home/ben/Downloads/output_baseline2.x_t')
with open('/home/ben/Downloads/match.json') as f:
    matches = json.load(f)

orig_part = p0
var_part = p1

features = PartFeatures()

orig_brep = part_to_graph(orig_part, features)
var_brep = part_to_graph(var_part, features)
export_id_hash = lambda x: xxhash.xxh32(x).intdigest()
hashed_matches = [(export_id_hash(match['val1']), export_id_hash(match['val2'])) for _,match in matches.items()]

index_dict = lambda x: dict((v.item(),k) for k,v in enumerate(x))
orig_face_map = index_dict(orig_brep.face_export_ids)
orig_edge_map = index_dict(orig_brep.edge_export_ids)
orig_vert_map = index_dict(orig_brep.vertex_export_ids)

var_face_map = index_dict(var_brep.face_export_ids)
var_edge_map = index_dict(var_brep.edge_export_ids)
var_vert_map = index_dict(var_brep.vertex_export_ids)

face_matches = []
edge_matches = []
vert_matches = []
for orig_id, var_id in hashed_matches:
    if orig_id in orig_face_map:
        assert(var_id in var_face_map)
        face_matches.append([orig_face_map[orig_id], var_face_map[var_id]])
    elif orig_id in orig_edge_map:
        assert(var_id in var_edge_map)
        edge_matches.append([orig_edge_map[orig_id], var_edge_map[var_id]])
    elif orig_id in orig_vert_map:
        assert(var_id in var_vert_map)
        vert_matches.append([orig_vert_map[orig_id], var_vert_map[var_id]])
    else:
        pass # The last match in our test wasn't in the dataset and was JFD, JFD -- special case?
        #assert(False) # Error - missing export id

face_matches = torch.tensor(face_matches).long().T if len(face_matches) > 0 else torch.empty((2,0)).long()
edge_matches = torch.tensor(edge_matches).long().T if len(edge_matches) > 0 else torch.empty((2,0)).long()
vert_matches = torch.tensor(vert_matches).long().T if len(vert_matches) > 0 else torch.empty((2,0)).long()

data = zip_hetdata(orig_brep, var_brep)
data.face_matches = face_matches
data.__edge_sets__['face_matches'] = ['left_faces', 'right_faces']
data.edge_matches = edge_matches
data.__edge_sets__['edge_matches'] = ['left_edges', 'right_edges']
data.vertex_matches = vert_matches
data.__edge_sets__['vertex_matches'] = ['left_vertices', 'right_vertices']


show_image(
    render_predictions(data, face_match_preds=data.face_matches)
    )